In [ ]:
!pip install torch torchvision --index-url https://download.pytorch.org/whl/cu117
!pip install transformers datasets timm

In [ ]:
import pandas as pd

# Load metadata
metadata_path = '/kaggle/input/chexpert/train.csv'
df = pd.read_csv(metadata_path)

# Display columns
print(df.columns)

In [ ]:
# Define label columns
label_columns = ['Cardiomegaly', 'Edema', 'Consolidation', 'Pneumonia', 'No Finding']

# Extract image paths and labels
df['Path'] = df['Path'].apply(lambda x: f"/kaggle/input/chexpert/{x}")  # Ensure correct path
labels = df[label_columns].fillna(0).astype(int)  # Replace NaNs and convert to integers


In [ ]:
df[label_columns] = df[label_columns].replace(-1, 0)


In [ ]:
df[label_columns] = df[label_columns].replace(-1, 1)


In [ ]:
df[label_columns]

In [ ]:
# Combine image paths and labels
dataset = pd.concat([df['Path'], labels], axis=1)



In [ ]:
dataset.head()


In [ ]:
import pandas as pd

# Load metadata
metadata_path = '/kaggle/input/chexpert/train.csv'
df = pd.read_csv(metadata_path)

# Ensure all label columns are numeric and replace invalid values
label_columns = ['Cardiomegaly', 'Edema', 'Consolidation', 'Pneumonia', 'No Finding']
df[label_columns] = df[label_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)

df['Path'] = df['Path'].str.replace(
    'CheXpert-v1.0-small/', '', regex=False
)  # Remove the incorrect prefix

# Ensure paths are correct for Kaggle directory structure
df['Path'] = df['Path'].apply(lambda x: f"/kaggle/input/chexpert/{x}")

# Combine paths and labels
dataset = pd.concat([df['Path'], df[label_columns]], axis=1)



In [ ]:

# Load metadata
metadata_path = '/kaggle/input/chexpert/valid.csv'
df = pd.read_csv(metadata_path)

# Ensure all label columns are numeric and replace invalid values
label_columns = ['Cardiomegaly', 'Edema', 'Consolidation', 'Pneumonia', 'No Finding']
df[label_columns] = df[label_columns].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)

df['Path'] = df['Path'].str.replace(
    'CheXpert-v1.0-small/', '', regex=False
)  # Remove the incorrect prefix

# Ensure paths are correct for Kaggle directory structure
df['Path'] = df['Path'].apply(lambda x: f"/kaggle/input/chexpert/{x}")

# Combine paths and labels
dataset_valid = pd.concat([df['Path'], df[label_columns]], axis=1)

In [ ]:
print(dataset.head())  # Preview the DataFrame

In [ ]:
print(dataset_valid.head())

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import torch

class CheXpertDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        
        # Load the image
        image_path = row['Path']
        image = Image.open(image_path).convert('RGB')
        
        # Convert labels to a float tensor
        label = torch.tensor(row[1:].values.astype(float), dtype=torch.float32)
        
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        
        return image, label



In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for ViT
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

transform_valid = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for ViT
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])



In [ ]:
train_dataset = CheXpertDataset(dataframe=dataset, transform=transform)
valid_dataset = CheXpertDataset(dataframe=dataset_valid, transform=transform)

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=2)

# Example: Iterate over the DataLoader
for images, labels in train_loader:
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    break

for images, labels in valid_loader:
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)
    break

In [ ]:
from transformers import ViTForImageClassification, ViTImageProcessor
from torch.optim import AdamW
import torch.nn as nn
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch

# Load pretrained ViT
model_name = "google/vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_name, num_labels=5, ignore_mismatched_sizes=True)  # Adjust num_labels

# Processor for ViT
processor = ViTImageProcessor.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=3e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=10)

# Loss function
criterion = nn.BCEWithLogitsLoss()


In [ ]:
def train_model(model, train_loader, valid_loader, optimizer, scheduler, criterion, device, epochs=5):
    scaler = torch.amp.GradScaler()
    model.to(device)

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")

        # Training Phase
        model.train()
        train_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            with torch.amp.autocast(device_type="cuda"):
                outputs = model(images)
                logits = outputs.logits if hasattr(outputs, "logits") else outputs  # Extract logits
                loss = criterion(logits, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

            train_loss += loss.item()

        # Print average training loss
        train_loss /= len(train_loader)
        print(f"Training Loss: {train_loss:.4f}")

        # Validation Phase
        model.eval()
        valid_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)

                with torch.amp.autocast(device_type="cuda"):
                    outputs = model(images)
                    logits = outputs.logits if hasattr(outputs, "logits") else outputs  # Extract logits
                    loss = criterion(logits, labels)

                valid_loss += loss.item()

                # Calculate accuracy (for multi-label, use a threshold)
                predictions = (torch.sigmoid(logits) > 0.5).float()  # Apply sigmoid for binary classification
                correct_predictions += (predictions == labels).sum().item()
                total_predictions += labels.numel()

        # Print average validation loss and accuracy
        valid_loss /= len(valid_loader)
        accuracy = correct_predictions / total_predictions * 100
        print(f"Validation Loss: {valid_loss:.4f}, Accuracy: {accuracy:.2f}%")

        # Step the scheduler (optional, if using learning rate scheduling)
        if scheduler is not None:
            scheduler.step(valid_loss)


In [ ]:
train_model(
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optimizer,
    scheduler=None,  # Pass `None` if not using a scheduler
    criterion=criterion,
    device=device,
    epochs=10
)

In [ ]:
model.save_pretrained("/kaggle/working/vit-chest-xray")
processor.save_pretrained("/kaggle/working/vit-chest-xray")


In [ ]:
!pip install huggingface-hub


In [ ]:
from huggingface_hub import notebook_login
notebook_login()


In [ ]:
from huggingface_hub import HfApi, HfFolder

# Define repository name (your Hugging Face username/repository)
repo_name = "vit-chest-xray"

# Upload to Hugging Face
model.push_to_hub(repo_name)
processor.push_to_hub(repo_name)


In [ ]:
!pip install datasets


In [ ]:
from huggingface_hub import notebook_login
notebook_login()